# Cleanup All Resources

Each notebook has a resource cleanup section that is designed to remove just the resources it created.

This notebook is designed to remove all resources by the type created.  By being a general cleanup utility, you can skip sections that you want to remain in use.  This notebook also has a section to delete the GCS bucket created in the `00 - Initial Setup` notebook.

**Resources**
- Vertex AI: https://googleapis.dev/python/aiplatform/latest/index.html
- BigQuery: https://googleapis.dev/python/bigquery/latest/index.html
- GCS: https://googleapis.dev/python/storage/latest/client.html

---
## Setup

Parameters:

In [6]:
PROJECT_ID = 'statmike-mlops'
REGION = 'us-central1'

PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

AI Platform Client Setup:

In [7]:
from google.cloud import aiplatform
from google.cloud import aiplatform_v1beta1

API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
client_options = {"api_endpoint": API_ENDPOINT}
clients = {}

aiplatform.init(project=PROJECT_ID, location=REGION)

PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

---
## BigQuery
- Delete the dataset `digits`

Make a client connection to BigQuery

In [95]:
from google.cloud import bigquery
clients['bq'] = bigquery.Client(project=PROJECT_ID)

list of datasets in the project:

In [96]:
datasets = list(clients['bq'].list_datasets())

delete each dataset:

In [ ]:
for ds in datasets:
    print(ds.dataset_id)
    clients['bq'].delete_dataset(ds.dataset_id,delete_contents=True,not_found_ok=True)

---
## Vertex AI > Datasets
- Delete datasets

In [14]:
datasets = aiplatform.TabularDataset.list()

In [15]:
for ds in datasets:
    print(ds.resource_name)
    aiplatform.TabularDataset(dataset_name=ds.resource_name).delete()

projects/691911073727/locations/us-central1/datasets/3655823385797066752
INFO:google.cloud.aiplatform.base:Deleting TabularDataset : projects/691911073727/locations/us-central1/datasets/3655823385797066752
INFO:google.cloud.aiplatform.base:Delete TabularDataset  backing LRO: projects/691911073727/locations/us-central1/operations/7698451388832940032
INFO:google.cloud.aiplatform.base:TabularDataset deleted. . Resource name: projects/691911073727/locations/us-central1/datasets/3655823385797066752


---
## Vertex AI > Features
- delete feature store including entity types and features

In [16]:
clients['fs'] = aiplatform_v1beta1.FeaturestoreServiceClient(client_options=client_options)

In [17]:
fs = clients['fs'].list_featurestores(parent=PARENT)

In [18]:
for f in fs:
    print(f.name)
    clients['fs'].delete_featurestore(request = aiplatform_v1beta1.types.DeleteFeaturestoreRequest(name=f.name,force=True))

projects/691911073727/locations/us-central1/featurestores/digits_featurestore


---
## Vertex AI > Pipelines
* Delete Managed Pipelines

In [19]:
pl = aiplatform.PipelineJob.list()

In [21]:
for p in pl:
    print(p.name)
    p.delete()

testname-20210908124552
INFO:google.cloud.aiplatform.base:Deleting PipelineJob : projects/691911073727/locations/us-central1/pipelineJobs/testname-20210908124552
INFO:google.cloud.aiplatform.base:Delete PipelineJob  backing LRO: projects/691911073727/locations/us-central1/operations/4275715672031363072
INFO:google.cloud.aiplatform.base:PipelineJob deleted. . Resource name: projects/691911073727/locations/us-central1/pipelineJobs/testname-20210908124552
hello-world-20210907225736
INFO:google.cloud.aiplatform.base:Deleting PipelineJob : projects/691911073727/locations/us-central1/pipelineJobs/hello-world-20210907225736
INFO:google.cloud.aiplatform.base:Delete PipelineJob  backing LRO: projects/691911073727/locations/us-central1/operations/1893311469152370688
INFO:google.cloud.aiplatform.base:PipelineJob deleted. . Resource name: projects/691911073727/locations/us-central1/pipelineJobs/hello-world-20210907225736


---
## Vertex AI > Training
- delete various jobs types
- some create with aiplatform.* and other created by aiplatform.gapic.*

## Vertex AI > Training > Training Pipelines
- Delete training pipelines

In [28]:
jobs = aiplatform.CustomTrainingJob.list()
for j in jobs:
    print(j.display_name)
    j.delete()

05C_AIP_DIGITS_20210907112707
INFO:google.cloud.aiplatform.base:Deleting CustomTrainingJob : projects/691911073727/locations/us-central1/trainingPipelines/6263963922928238592
INFO:google.cloud.aiplatform.base:Delete CustomTrainingJob  backing LRO: projects/691911073727/locations/us-central1/operations/3760053514697441280
INFO:google.cloud.aiplatform.base:CustomTrainingJob deleted. . Resource name: projects/691911073727/locations/us-central1/trainingPipelines/6263963922928238592
05C_AIP_DIGITS_20210902141249
INFO:google.cloud.aiplatform.base:Deleting CustomTrainingJob : projects/691911073727/locations/us-central1/trainingPipelines/7521963554778382336
INFO:google.cloud.aiplatform.base:Delete CustomTrainingJob  backing LRO: projects/691911073727/locations/us-central1/operations/6831508460564119552
INFO:google.cloud.aiplatform.base:CustomTrainingJob deleted. . Resource name: projects/691911073727/locations/us-central1/trainingPipelines/7521963554778382336
05C_AIP_DIGITS_20210902134247
INFO

In [29]:
jobs = aiplatform.CustomPythonPackageTrainingJob.list()
for j in jobs:
    print(j.display_name)
    j.delete()

In [30]:
jobs = aiplatform.AutoMLTabularTrainingJob.list()
for j in jobs:
    print(j.display_name)
    j.delete()

job-02c-automl_20210908124535
INFO:google.cloud.aiplatform.base:Deleting AutoMLTabularTrainingJob : projects/691911073727/locations/us-central1/trainingPipelines/521029973100724224
INFO:google.cloud.aiplatform.base:Delete AutoMLTabularTrainingJob  backing LRO: projects/691911073727/locations/us-central1/operations/4318499868491382784
INFO:google.cloud.aiplatform.base:AutoMLTabularTrainingJob deleted. . Resource name: projects/691911073727/locations/us-central1/trainingPipelines/521029973100724224


## Vertex AI > Training > Custom Job
- Delete custom training jobs

In [32]:
jobs = aiplatform.CustomJob.list()
for j in jobs:
    print(j.display_name)
    j.delete()

caip_pipelines_3056334461961502720_7553741639844364288
INFO:google.cloud.aiplatform.base:Deleting CustomJob : projects/691911073727/locations/us-central1/customJobs/5817263134888427520
INFO:google.cloud.aiplatform.base:Delete CustomJob  backing LRO: projects/691911073727/locations/us-central1/operations/5714615752976236544
INFO:google.cloud.aiplatform.base:CustomJob deleted. . Resource name: projects/691911073727/locations/us-central1/customJobs/5817263134888427520
caip_pipelines_3056334461961502720_-6281316415437799424
INFO:google.cloud.aiplatform.base:Deleting CustomJob : projects/691911073727/locations/us-central1/customJobs/5187322137009979392
INFO:google.cloud.aiplatform.base:Delete CustomJob  backing LRO: projects/691911073727/locations/us-central1/operations/3408772743762542592
INFO:google.cloud.aiplatform.base:CustomJob deleted. . Resource name: projects/691911073727/locations/us-central1/customJobs/5187322137009979392
caip_pipelines_3056334461961502720_-1669630397010411520
INF

## Vertex AI > Training > Hyperparameter Tuning Job

In [34]:
jobs = aiplatform.HyperparameterTuningJob.list()
for j in jobs:
    print(j.display_name)
    j.delete()

05E2_AIP_DIGITS_20210907152238
INFO:google.cloud.aiplatform.base:Deleting HyperparameterTuningJob : projects/691911073727/locations/us-central1/hyperparameterTuningJobs/6535024325500600320
INFO:google.cloud.aiplatform.base:Delete HyperparameterTuningJob  backing LRO: projects/691911073727/locations/us-central1/operations/5921781335835279360
INFO:google.cloud.aiplatform.base:HyperparameterTuningJob deleted. . Resource name: projects/691911073727/locations/us-central1/hyperparameterTuningJobs/6535024325500600320
05E2_AIP_DIGITS_20210907151121
INFO:google.cloud.aiplatform.base:Deleting HyperparameterTuningJob : projects/691911073727/locations/us-central1/hyperparameterTuningJobs/4409325301381726208
INFO:google.cloud.aiplatform.base:Delete HyperparameterTuningJob  backing LRO: projects/691911073727/locations/us-central1/operations/3615938326621585408
INFO:google.cloud.aiplatform.base:HyperparameterTuningJob deleted. . Resource name: projects/691911073727/locations/us-central1/hyperparamete

---
## Vertex AI > Experiments > Experiment & Tensorboard Instances
ISSUE: remove experiments first then instances.

In [43]:
tb = aiplatform.Tensorboard.list()
for t in tb:
    print(t)
    t.delete()

resource name: projects/691911073727/locations/us-central1/tensorboards/3436703912520843264
INFO:google.cloud.aiplatform.base:Deleting Tensorboard : projects/691911073727/locations/us-central1/tensorboards/3436703912520843264
INFO:google.cloud.aiplatform.base:Delete Tensorboard  backing LRO: projects/691911073727/locations/us-central1/operations/1513601725569695744
INFO:google.cloud.aiplatform.base:Tensorboard deleted. . Resource name: projects/691911073727/locations/us-central1/tensorboards/3436703912520843264


In [44]:
clients['tb'] = aiplatform_v1beta1.TensorboardServiceClient(client_options=client_options)

In [45]:
tbs = clients['tb'].list_tensorboards(parent=PARENT)

In [46]:
for tb in tbs:
    print(tb.name)
    tbes = clients['tb'].list_tensorboard_experiments(parent=tb.name)
    for tbe in tbes:
        print(tbe.name)
        clients['tb'].delete_tensorboard_experiment(request=aiplatform_v1beta1.types.DeleteTensorboardExperimentRequest(name=tbe.name))
    clients['tb'].delete_tensorboard(request = aiplatform_v1beta1.types.DeleteTensorboardRequest(name=tb.name))

## Vertex AI > Experiments > Studies
Vizier Studies

In [49]:
clients['viz'] = aiplatform_v1beta1.VizierServiceClient(client_options=client_options)

In [50]:
studies = clients['viz'].list_studies(parent=PARENT)

In [51]:
for study in studies:
    print(study.name)
    clients['viz'].delete_study(name=study.name)

projects/691911073727/locations/us-central1/studies/2706485868636
projects/691911073727/locations/us-central1/studies/3240081905661


---
## Vertex AI > Endpoints
- Delete endpoints forcing deployed models to undeploy

In [8]:
endpoints = aiplatform.Endpoint.list()

In [9]:
for ep in endpoints:
    print(ep.resource_name)
    #ep.delete(force=True)

projects/691911073727/locations/us-central1/endpoints/7199267887142404096
projects/691911073727/locations/us-central1/endpoints/8262117399201841152


---
## Vertex AI > Models
- Delete endpoints first (contain deployed models)
- Delete uploaded models

In [57]:
models = aiplatform.Model.list()

In [58]:
for model in models:
    print(model.resource_name)
    model.delete()

projects/691911073727/locations/us-central1/models/5663478841557909504
INFO:google.cloud.aiplatform.base:Deleting Model : projects/691911073727/locations/us-central1/models/5663478841557909504
INFO:google.cloud.aiplatform.base:Delete Model  backing LRO: projects/691911073727/locations/us-central1/operations/910119375502049280
INFO:google.cloud.aiplatform.base:Model deleted. . Resource name: projects/691911073727/locations/us-central1/models/5663478841557909504
projects/691911073727/locations/us-central1/models/3616592810918019072
INFO:google.cloud.aiplatform.base:Deleting Model : projects/691911073727/locations/us-central1/models/3616592810918019072
INFO:google.cloud.aiplatform.base:Delete Model  backing LRO: projects/691911073727/locations/us-central1/operations/1337961340102246400
INFO:google.cloud.aiplatform.base:Model deleted. . Resource name: projects/691911073727/locations/us-central1/models/3616592810918019072
projects/691911073727/locations/us-central1/models/776623765507486515

---
## Vertex AI > Batch Predictions
- delete all batch prediction jobs

In [59]:
bps = aiplatform.BatchPredictionJob.list()

In [60]:
for bp in bps:
    print(bp.resource_name)
    bp.delete()

projects/691911073727/locations/us-central1/batchPredictionJobs/1358699503791636480
INFO:google.cloud.aiplatform.base:Deleting BatchPredictionJob : projects/691911073727/locations/us-central1/batchPredictionJobs/1358699503791636480
INFO:google.cloud.aiplatform.base:Delete BatchPredictionJob  backing LRO: projects/691911073727/locations/us-central1/operations/4670625064356413440
INFO:google.cloud.aiplatform.base:BatchPredictionJob deleted. . Resource name: projects/691911073727/locations/us-central1/batchPredictionJobs/1358699503791636480


---
## Vertex AI > Metadata

In [82]:
clients['md'] = aiplatform_v1beta1.MetadataServiceClient(client_options=client_options)

In [83]:
md = clients['md'].list_metadata_stores(parent=PARENT)

In [88]:
for m in md:
    print(m.name)
    arts = clients['md'].list_artifacts(parent=m.name)
    for a in arts:
        print(a)
        clients['md'].delete_artifact(name=a.name)
    #clients['md'].get_metadata_store(name=m.name)#.purge_artifacts(parent=m.name)

projects/691911073727/locations/us-central1/metadataStores/default
name: "projects/691911073727/locations/us-central1/metadataStores/default/artifacts/11084318035124877310"
display_name: "endpoint"
uri: "aiplatform://v1/projects/691911073727/locations/us-central1/endpoints/5446945419630215168"
etag: "1631109713213"
create_time {
  seconds: 1631108707
  nanos: 47000000
}
update_time {
  seconds: 1631109713
  nanos: 213000000
}
state: LIVE
schema_title: "system.Artifact"
schema_version: "0.0.1"
metadata {
}

name: "projects/691911073727/locations/us-central1/metadataStores/default/artifacts/7505302604043269173"
display_name: "model"
uri: "aiplatform://v1/projects/691911073727/locations/us-central1/models/5663478841557909504"
etag: "1631108705974"
create_time {
  seconds: 1631105335
  nanos: 671000000
}
update_time {
  seconds: 1631108705
  nanos: 974000000
}
state: LIVE
schema_title: "system.Model"
schema_version: "0.0.1"
metadata {
}

name: "projects/691911073727/locations/us-central1/m

---
## Storage Bucket Removal

- delete contents and storage bucket `statmike-mlops` (named after project)

In [3]:
from google.cloud import storage
clients['gcs'] = storage.Client()

In [4]:
bucket = clients['gcs'].get_bucket(PROJECT_ID)

In [5]:
bucket.delete(force=True)